<a href="https://colab.research.google.com/github/alvarofpinheiro/pifwia_ga/blob/main/PIFWIA_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Algoritmo Genético em Inteligência Computacional é uma metaheurística inspirada no processo de seleção natural que pertence à classe maior de algoritmos evolutivos (EA). Os algoritmos genéticos são comumente usados para gerar soluções de alta qualidade para problemas de otimização e busca, contando com operadores biologicamente inspirados, como mutação, cruzamento e seleção. Alguns exemplos de aplicações GA incluem a otimização de árvores de decisão para melhor desempenho, resolução automática de quebra-cabeças de sudoku, otimização de hiperparâmetros, etc.

Whitley, D. A genetic algorithm tutorial. Statistics and Computing. 4 (2): 65–85. CiteSeerX 10.1.1.184.3999. S2CID 3447126. https://doi.org/10.1007/BF00175354, 1994.

Pseudo-código:

```
populacao_inicial = gerar_populacao()

while(criterio de parada):
  while(numero de pais < populacao*2):
    # decide quais individuos serão utilizados para gerar uma nova geração
    selecionar_pais()

  for each(par de pais):
    # a partir da combinação dos pais gera novos individuos
    realizar_crossover()

  for each(individuo):
    # realiza mudancas nos individuos para incentivar exploração
    realizar_mutacao()

  for each(individuo):
    # verificar qualidade dos individuos
    calcular_fitness()
```

In [ ]:
# Bibliotecas
import random
import math
import matplotlib.pyplot as plt

In [ ]:
# Caixeiro Viajante
def gerar_grafico(lista_cidades, solucao, titulo):
  x = []
  y = []
  for i in range(0, NUMERO_DE_CIDADES):
    x.append(lista_cidades[solucao[i]].posicao_x)
    y.append(lista_cidades[solucao[i]].posicao_y)
    plt.plot(lista_cidades[solucao[i]].posicao_x, 
             lista_cidades[solucao[i]].posicao_y, 'ro')

  x.append(lista_cidades[solucao[0]].posicao_x)
  y.append(lista_cidades[solucao[0]].posicao_y)
  plt.plot(x, y)
  plt.title(titulo)
  plt.show()

In [ ]:
# Mostra a mudanca do melhor fitness pelas iteracoes
def gerar_grafico_desempenho_tempo(fitness):
  x = []
  y = []
  for i in range(len(fitness)):
    x.append(i)
    y.append(fitness[i])

  plt.plot(x, y)
  plt.title('Desempenho X Tempo')
  plt.show()

In [ ]:
# Codigo para o problema do caixeiro viajante
POSICOES = [
[ 11003.611100, 42102.500000],
[ 11108.611100, 42373.888900],
[ 11133.333300, 42885.833300],
[ 11155.833300, 42712.500000],
[ 11183.333300, 42933.333300],
[ 11297.500000, 42853.333300],
[ 11310.277800, 42929.444400],
[ 11416.666700, 42983.333300],
[ 11423.888900, 43000.277800],
[ 11438.333300, 42057.222200],
[ 11461.111100, 43252.777800],
[ 11485.555600, 43187.222200],
[ 11503.055600, 42855.277800],
[ 11511.388900, 42106.388900],
[ 11522.222200, 42841.944400],
[ 11569.444400, 43136.666700],
[ 11583.333300, 43150.000000],
[ 11595.000000, 43148.055600],
[ 11600.000000, 43150.000000],
[ 11690.555600, 42686.666700],
[ 11715.833300, 41836.111100],
[ 11751.111100, 42814.444400],
[ 11770.277800, 42651.944400],
[ 11785.277800, 42884.444400],
[ 11822.777800, 42673.611100],
[ 11846.944400, 42660.555600],
[ 11963.055600, 43290.555600],
[ 11973.055600, 43026.111100],
[ 12058.333300, 42195.555600],
[ 12149.444400, 42477.500000],
[ 12286.944400, 43355.555600],
[ 12300.000000, 42433.333300],
[ 12355.833300, 43156.388900],
[ 12363.333300, 43189.166700],
[ 12372.777800, 42711.388900],
[ 12386.666700, 43334.722200],
[ 12421.666700, 42895.555600],
[ 12645.000000, 42973.333300]]

NUMERO_DE_CIDADES = len(POSICOES)

class cidade:
  def __init__(self, x, y):
    self.posicao_x = x
    self.posicao_y = y
  def distancia(self, cidade_externa):
    return  math.sqrt((self.posicao_x - cidade_externa.posicao_x)**2 + 
                      (self.posicao_y - cidade_externa.posicao_y)**2)

def instanciando_cidades():
  lista = []
  for i in range(0, NUMERO_DE_CIDADES):
    lista.append(cidade(POSICOES[i][0], POSICOES[i][1]))
  return lista

def gerar_solucao_aleatoria():
  lista_de_cidades = list(range(0, NUMERO_DE_CIDADES))
  lista_aleatoria = []
  for i in range(0, NUMERO_DE_CIDADES):
    cidade = random.choice(lista_de_cidades)
    lista_aleatoria.append(cidade)
    lista_de_cidades.remove(cidade)
  return lista_aleatoria

def calcular_fitness(possivel_solucao):
  fitness = 0
  for i in range(0, NUMERO_DE_CIDADES): 
    if i < NUMERO_DE_CIDADES - 1:
      fitness += cidades[possivel_solucao[i]].distancia(cidades[possivel_solucao[i+1]])
    else:
      fitness += cidades[possivel_solucao[i]].distancia(cidades[possivel_solucao[0]])
  return 1/fitness

In [ ]:
# Hiperparâmetros Algoritmo Genetico
ITERACOES = 2000
POPULACAO = 50
TORNEIO_PROB = 0.4
PROB_MUTACAO = 0.1

In [ ]:
# Mutação
def mutacao( individuo):
  if random.random() < PROB_MUTACAO:
    individuo_mutado = []
    aux_1 = random.randint(0, NUMERO_DE_CIDADES-1)
    aux_2 = random.randint(0, NUMERO_DE_CIDADES-1)
    while(aux_1 == aux_2):
      aux_2 = random.randint(0, NUMERO_DE_CIDADES-1)
    if aux_2 < aux_1:
      aux_3 = aux_1
      aux_1 = aux_2
      aux_2 = aux_3
    for i in range(aux_1):
      individuo_mutado.append(individuo[i])
    for i in range(aux_2-1, aux_1-1, -1):
      individuo_mutado.append(individuo[i])
    for i in range(aux_2, len(individuo)):
      individuo_mutado.append(individuo[i])
    return individuo_mutado
  return individuo

In [ ]:
# Crossover
def crossover( individuos, pais):
  aux_1 = random.randint(0, NUMERO_DE_CIDADES-1)
  aux_2 = random.randint(0, NUMERO_DE_CIDADES-1)
  if aux_2 < aux_1:
    aux_3 = aux_1
    aux_1 = aux_2
    aux_2 = aux_3
  list_pai = individuos[pais[0]][aux_1:aux_2]
  filho = []
  i = 0
  while len(filho) < aux_1:
    if individuos[pais[1]][i] not in list_pai:
      filho.append(individuos[pais[1]][i])
    i+=1
  filho = filho + list_pai
  i = 0
  while len(filho) < NUMERO_DE_CIDADES:
    if individuos[pais[1]][i] not in filho:
      filho.append(individuos[pais[1]][i])
    i+=1
  return filho

In [ ]:
# Seleção
def selecao_pais(fitness):
  pais = []
  for i in range(POPULACAO):
    pais.append(torneio(fitness))
  return pais

def torneio(fitness):
  pais = []
  for k in range(0, 2):
    individuos_torneio = []
    for i in range(POPULACAO):
      aux = random.random()
      if aux < TORNEIO_PROB:
        individuos_torneio.append(i)
    while len(individuos_torneio) < 2:
      individuos_torneio.append(random.randint(0, POPULACAO-1))
    best = fitness[individuos_torneio[0]]
    best_ind = individuos_torneio[0]
    for i in range(len(individuos_torneio)):
      if fitness[individuos_torneio[i]] > best:
        best = fitness[individuos_torneio[i]]
        best_ind = individuos_torneio[i]
    pais.append(best_ind)

  return pais

In [ ]:
# Execução do GA
cidades = instanciando_cidades()

In [ ]:
# Inicializando os individuos
individuos = []
fitness = []
fitness_tempo = []
for i in range(POPULACAO):
  individuos.append(gerar_solucao_aleatoria())
  fitness.append(calcular_fitness(individuos[-1]))

print('Distancia inicial: ' , 1/fitness[0])
gerar_grafico(cidades, individuos[0], 'Solução aleatória inicial')

best = 0
best_route = []
for i in range(ITERACOES):
  pais = selecao_pais(fitness)
  filhos = []
  for j in pais:
    filhos.append(crossover(individuos ,j))
  individuos = filhos
  for k in range(POPULACAO):
    individuos[k] = mutacao(individuos[k])
  fitness = []
  for k in range(POPULACAO):
    fitness.append(calcular_fitness(individuos[k]))
    if fitness[-1] > best:
      best = fitness[-1]
      best_route = individuos[k]
  fitness_tempo.append(1/best)

In [ ]:
# Plotando

print('Melhor: ' , 1/best)
gerar_grafico(cidades, best_route, 'Melhor Solução')
gerar_grafico_desempenho_tempo(fitness_tempo)